---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [11]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    p_tag = nltk.pos_tag(nltk.word_tokenize(doc))

    result = []
    for i,j in p_tag:
        if(wn.synsets(i,convert_tag(j))==[]):
            continue
        result.append(wn.synsets(i,convert_tag(j))[0])
    
    return result

def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    max_collection = []
    for i in s1:
        house = []
        for j in s2:
            if(i.path_similarity(j)==None): continue
            house.append(i.path_similarity(j))
            
        max_collection.append(max(house,default=0))
    
    count=0
    for i in max_collection:
        if(i==0):
            continue
        else:
            count+=1
    #print(max_collection)
    return sum(max_collection)/count

def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2


### test_document_path_similarity

Use this function to check if doc_to_synsets and similarity_score are correct.

*This function should return the similarity score as a float.*

In [183]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

test_document_path_similarity()

0.554265873015873

<br>
___
`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [2]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [240]:
def most_similar_docs():
    score_list=[]
    for i in range(len(paraphrases)):
        score_list.append(document_path_similarity(paraphrases["D1"].iloc[i],paraphrases["D2"].iloc[i]))
    return (paraphrases['D1'].iloc[np.argmax(score_list)],paraphrases['D2'].iloc[np.argmax(score_list)],max(score_list))
most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.9753086419753086)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [260]:
def label_accuracy():
    from sklearn.metrics import accuracy_score
    score_list = []
    for i in range(len(paraphrases)):
        score_list.append(document_path_similarity(paraphrases["D1"].iloc[i],paraphrases["D2"].iloc[i]))
    paraphrases['score'] = score_list
    paraphrases['label'] = 1
    paraphrases['label'].where(paraphrases['score']>0.75,0,inplace=True)
    return accuracy_score(paraphrases['Quality'],paraphrases['label'])
    
    
label_accuracy()

0.80000000000000004

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [19]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')

# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())



In [4]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus,passes=25,random_state=34,id2word=id_map)

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [272]:
def lda_topics():
    
    return ldamodel.print_topics(num_topics=10,num_words=10)

lda_topics()

[(76,
  '0.090*"data" + 0.028*"price" + 0.023*"use" + 0.022*"products" + 0.020*"fax" + 0.019*"include" + 0.018*"areas" + 0.018*"included" + 0.017*"break" + 0.016*"week"'),
 (78,
  '0.047*"theory" + 0.028*"did" + 0.028*"know" + 0.027*"extra" + 0.025*"don" + 0.023*"came" + 0.022*"got" + 0.022*"original" + 0.021*"new" + 0.021*"word"'),
 (46,
  '0.109*"mac" + 0.021*"new" + 0.021*"cheaper" + 0.018*"market" + 0.016*"buy" + 0.016*"pain" + 0.015*"removed" + 0.013*"appreciated" + 0.013*"years" + 0.013*"sell"'),
 (28,
  '0.098*"different" + 0.051*"things" + 0.041*"key" + 0.041*"thank" + 0.039*"bad" + 0.033*"people" + 0.031*"digital" + 0.030*"mail" + 0.029*"wouldn" + 0.022*"written"'),
 (96,
  '0.040*"won" + 0.039*"right" + 0.025*"year" + 0.025*"got" + 0.024*"1992" + 0.021*"didn" + 0.021*"fact" + 0.017*"feet" + 0.016*"did" + 0.015*"don"'),
 (44,
  '0.036*"red" + 0.022*"time" + 0.022*"blue" + 0.020*"players" + 0.018*"said" + 0.017*"pain" + 0.016*"video" + 0.015*"told" + 0.015*"got" + 0.014*"oil"')

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [276]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [326]:
def topic_distribution():
    #vectorize the new doc.
    vect = CountVectorizer()
    XX = vect.fit_transform(new_doc)
    new_corpus = gensim.matutils.Sparse2Corpus(XX,documents_columns= False)
    
    # in order to get the probability distribution, use the function get_document_topics(new_corpus) onto the model that we previously made
    result = list(ldamodel.get_document_topics(new_corpus))[0]
    return result
topic_distribution()

[(4, 0.086206499102573095),
 (11, 0.066742611041041636),
 (26, 0.034380360721703071),
 (54, 0.22559632761806123),
 (66, 0.065139520893542371),
 (70, 0.065864325790555109),
 (74, 0.16805497263594518),
 (77, 0.12767042626443431),
 (90, 0.073032580122803017),
 (99, 0.066857830354796075)]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [1]:
def topic_names():
    topicc =  ['Business', 'Education', 'Computers & IT', 'Religion', 'Politics', 'Sports', 'Health', 
               'Automobiles', 'Science', 'Society & Lifestyle']
    #subjective matter so based on the given topics you found, just label the topics as you want among the topics given above.
    return topicc
topic_names()

['Business',
 'Education',
 'Computers & IT',
 'Religion',
 'Politics',
 'Sports',
 'Health',
 'Automobiles',
 'Science',
 'Society & Lifestyle']